In [10]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interactive


In [21]:
POPINDEX_CURRENT = {
    "FRAGRANCE": {
      "vAddedToProfile": 0.2 ,
      "vLiked": 0.2 ,
           "vBuy": 0.2 ,
              "vTry": 0.2 ,
              "vShared": 0.2,
     },
    "BRAND": {
            "vAddedToProfile": 0.4 ,
            "vLiked": 0.3 ,
            "vShared": 0.3 
    },

    "FAMILY": {
        "vAddedToProfile": 0.2 ,
      "vLiked": 0.2 ,
           "vBuy": 0.2 ,
              "vTry": 0.2 ,
              "vShared": 0.2,
    },
    "INGREDIENT": {
        "vSearch": 1.0
    }
}

In [11]:
db = pd.read_excel('output-final-products.xlsx')

In [12]:
db['sensor'] = pd.Series(db.sensor).fillna(method='ffill')
db['entity'] = pd.Series(db.entity).fillna(method='ffill')
db = db.set_index(['sensor', 'entity', 'kpi'])
def evaluate(x):
    if type(x)==str:
        return pd.Series(eval(x))
    return x
db = db.applymap(evaluate)

In [13]:
def createDictFromPandas(df):
    if (df.index.nlevels==1):
        return df
    dict_f = {}
    for level in df.index.levels[0]:
        if (level in df.index):
            dict_f[level] = createDictFromPandas(df.xs([level]))
    return dict_f


In [14]:
def displayTop(series_g):
    
    return widgets.interactive(lambda : display(series_g.sort_values(ascending=False).head(10)))

In [23]:
def displayCollectionKPIs(dict_kpis, entity):
    list_kpis = []
    dict_values = {}
    kpis = []
    currentPoP = None
    for k,v in dict_kpis.items():
        if ('total' in k.lower() or 'interaction' in k.lower()):
            pass
        elif (k[0]=='v'):
            title = widgets.HTML(
                value="<b>{} </b>({})".format(k,dict_kpis[k+'Total']),
            )
#             print(k,v)
            topKpi = displayTop(v)
#             display(topKpi)
            # get initial value based on entity
#             slider = 
#             print(topKpi, slider)
            vista = widgets.VBox([title, topKpi])
            if ('popindex' in k.lower()):
                currentPoP = vista
            else:
                list_kpis.append(vista)
                kpis.append(k)
            # create slider
    kwargs = {k:widgets.FloatSlider(value=POPINDEX_CURRENT[entity][k] if k in POPINDEX_CURRENT[entity] else 0, min=0, max=1, step=0.01,readout=True) for k in kpis}
    def getPoPIndex(weights):
        try:
            return pd.concat([dict_kpis[k]*w if not pd.isnull(w) and k in dict_kpis else pd.Series() for k,w in weights.items()]).groupby(level=0).sum().sort_values(ascending=False).head(10)
        except:
            return ''
    def displayInteractive(**kwargs):
        try:
            display(getPoPIndex(kwargs))
        except:
            pass
    interactiveStack = interactive(
#             lambda **kwargs: display(widgets.HTML(value="<b>{}</b>".format(str(kwargs)))), 
            displayInteractive, 
            *[],
            **kwargs
    )
    panelTop = widgets.HBox(list_kpis)
    # display currant popIndex
    # & create fake popIndex
    panelBottom = widgets.HBox([currentPoP, widgets.VBox([widgets.HTML(value="<b>Simulated PoP</b>"),interactiveStack])])
    return widgets.VBox([panelTop, panelBottom])

In [24]:
# displayCollectionKPIs(db.loc['douglas_profiler','BRAND'][pd.Timestamp('2022-01-19')].to_dict(), "BRAND")

In [25]:
def createAccordionFromPandas(df, key=None):
    if (df.index.nlevels==1):
        return displayCollectionKPIs(df.to_dict(), key)
    children = []
    titles = []
    for level in df.index.levels[0]:
        if (level in df.index):
            child = createAccordionFromPandas(df.xs([level]), level)
            children.append(child)
            titles.append(level)
    accordion = widgets.Accordion(children=children, titles=tuple(titles))
    [accordion.set_title(i,t) for i,t in enumerate(titles)]
    return accordion

In [26]:
import datetime

In [28]:
def displayAccordion(column_date):
    try:
        return display(createAccordionFromPandas(db[pd.Timestamp(column_date)]))
    except Exception as ex:
        print(ex)
        pass

widgets.VBox([
widgets.HTML(value="<h1>PoP Lab</h1>"
widgets.interactive(
    displayAccordion,
    column_date=widgets.DatePicker(
        description='Pick a Date',
        disabled=False,
        value = datetime.date(2022,1,23)
    )
)])

SyntaxError: invalid syntax (<ipython-input-28-0a405d406a6b>, line 10)